In [5]:
import pandas as pd
import numpy as np

# Load GNSS data from file
# Replace 'gnss_data.csv' with your actual file path
gnss_data = pd.read_csv('gnss_dataset.csv')

# Display basic information about the dataset
print("GNSS Data Shape:", gnss_data.shape)
print("\nColumn Names:")
print(gnss_data.columns.tolist())
print("\nFirst few rows:")
print(gnss_data.head())

GNSS Data Shape: (300, 3)

Column Names:
['timestamp', 'latitude', 'longitude']

First few rows:
                    timestamp   latitude   longitude
0  2025-05-28 17:29:06.807800  37.774924 -122.419367
1  2025-05-28 17:29:07.807800  37.774957 -122.419324
2  2025-05-28 17:29:08.807800  37.774980 -122.419309
3  2025-05-28 17:29:09.807800  37.775002 -122.419281
4  2025-05-28 17:29:10.807800  37.775022 -122.419251


In [6]:
from geopy.distance import geodesic
from datetime import datetime
from math import atan2, degrees, radians, cos, sin

# Convert timestamp to datetime objects
gnss_data['timestamp'] = pd.to_datetime(gnss_data['timestamp'])

# Initialize speed list
speeds = [0]  # First point has speed 0

# Calculate speed for each subsequent point
for i in range(1, len(gnss_data)):
    # Get current and previous coordinates
    prev_point = (gnss_data.iloc[i-1]['latitude'], gnss_data.iloc[i-1]['longitude'])
    curr_point = (gnss_data.iloc[i]['latitude'], gnss_data.iloc[i]['longitude'])
    
    # Calculate distance in meters
    distance = geodesic(prev_point, curr_point).meters
    
    # Calculate time difference in seconds
    time_diff = (gnss_data.iloc[i]['timestamp'] - gnss_data.iloc[i-1]['timestamp']).total_seconds()
    
    # Calculate speed in m/s
    if time_diff > 0:
        speed_ms = distance / time_diff
    else:
        speed_ms = 0
    
    speeds.append(speed_ms)

# Add speed column to dataframe
gnss_data['speed_ms'] = speeds
gnss_data['speed_kmh'] = gnss_data['speed_ms'] * 3.6  # Convert to km/h


print("Speed calculation completed!")
print(f"Average speed: {gnss_data['speed_ms'].mean():.2f} m/s ({gnss_data['speed_kmh'].mean():.2f} km/h)")
print(f"Max speed: {gnss_data['speed_ms'].max():.2f} m/s ({gnss_data['speed_kmh'].max():.2f} km/h)")
# Calculate bearing from start to end position

def calculate_bearing(lat1, lon1, lat2, lon2):
    """
    Calculate the bearing between two points on the earth (specified in decimal degrees)
    """
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlon = lon2 - lon1
    
    y = sin(dlon) * cos(lat2)
    x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    
    bearing = atan2(y, x)
    bearing = degrees(bearing)
    bearing = (bearing + 360) % 360  # Normalize to 0-360 degrees
    
    return bearing

# Get start and end positions
start_lat = gnss_data.iloc[0]['latitude']
start_lon = gnss_data.iloc[0]['longitude']
end_lat = gnss_data.iloc[-1]['latitude']
end_lon = gnss_data.iloc[-1]['longitude']

# Calculate bearing from start to end
bearing = calculate_bearing(start_lat, start_lon, end_lat, end_lon)

print(f"\nBearing from start to end position: {bearing:.2f} degrees")
print(f"Start position: ({start_lat:.6f}, {start_lon:.6f})")
print(f"End position: ({end_lat:.6f}, {end_lon:.6f})")



Speed calculation completed!
Average speed: 3.40 m/s (12.25 km/h)
Max speed: 6.18 m/s (22.24 km/h)

Bearing from start to end position: 45.15 degrees
Start position: (37.774924, -122.419367)
End position: (37.781082, -122.411534)


In [7]:
print("\nFirst 10 rows with speed:")
print(gnss_data[['timestamp', 'latitude', 'longitude', 'speed_ms', 'speed_kmh']].head(10))


First 10 rows with speed:
                   timestamp   latitude   longitude  speed_ms  speed_kmh
0 2025-05-28 17:29:06.807800  37.774924 -122.419367  0.000000   0.000000
1 2025-05-28 17:29:07.807800  37.774957 -122.419324  5.242171  18.871816
2 2025-05-28 17:29:08.807800  37.774980 -122.419309  2.922430  10.520749
3 2025-05-28 17:29:09.807800  37.775002 -122.419281  3.427395  12.338621
4 2025-05-28 17:29:10.807800  37.775022 -122.419251  3.408644  12.271118
5 2025-05-28 17:29:11.807800  37.775051 -122.419221  4.264023  15.350484
6 2025-05-28 17:29:12.807800  37.775077 -122.419199  3.427527  12.339095
7 2025-05-28 17:29:13.807800  37.775105 -122.419171  3.994338  14.379616
8 2025-05-28 17:29:14.807800  37.775127 -122.419126  4.604487  16.576154
9 2025-05-28 17:29:15.807800  37.775154 -122.419079  5.120598  18.434152


In [8]:
import folium

# Create a map centered on the mean coordinates
center_lat = gnss_data['latitude'].mean()
center_lon = gnss_data['longitude'].mean()

# Create the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Create a list of coordinates for the line
coordinates = [[row['latitude'], row['longitude']] for idx, row in gnss_data.iterrows()]

# Add a polyline connecting all points
folium.PolyLine(
    locations=coordinates,
    color='red',
    weight=2,
    opacity=0.8
).add_to(m)

# Display the map
m